In [1]:
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
import langchain

In [2]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    temperature = 0.9,
    groq_api_key = "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"
)

In [3]:
response = llm.invoke("Write a log of 700 words on AI")

print(response.content)

**Artificial Intelligence (AI) Log: Understanding the Future of Technology**

As we continue to advance in the realm of technology, it has become increasingly evident that Artificial Intelligence (AI) is playing a pivotal role in shaping our world. From simple chatbots to complex machines capable of learning and decision-making, AI has invaded every aspect of our lives. In this log, we will delve into the world of AI, exploring its history, applications, and the impact it has on our society.

**A Brief History of AI**

The concept of AI dates back to the mid-20th century, when computer scientists like Alan Turing, Marvin Minsky, and John McCarthy began exploring the idea of creating intelligent machines. The term "Artificial Intelligence" was first coined in 1956, during a conference at Dartmouth College. Over the years, AI has evolved from simple rule-based systems to complex machine learning algorithms that enable machines to learn from experience.

**Applications of AI**

AI has num

In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(urls=[ "https://codeforces.com/problemset/problem/2077/G"])
doc = loader.load()

In [15]:
print(doc)
print(doc[0])

[Document(metadata={'source': 'https://codeforces.com/problemset/problem/2077/G'}, page_content='')]
page_content='' metadata={'source': 'https://codeforces.com/problemset/problem/2077/G'}


In [7]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("Give the description about this scrapped data : {data}")

chain = template | llm

In [8]:
output = chain.invoke(input={"data": doc[0].page_content[:10000]})

In [9]:
print(output.content)

There is no scrapped data provided for me to describe. Please provide the data you'd like me to describe, and I'll be happy to help.


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

text = text_splitter.split_documents(doc)

In [9]:
print(len(text))

116


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:

model = embeddings.client

model.save("./models/all-MiniLM-L6-v2")
print("Model saved successfully!")


Model saved successfully!


In [12]:
model = HuggingFaceEmbeddings(model_name="./models/all-MiniLM-L6-v2")

C:\Users\Apurav\AppData\Local\Temp\ipykernel_39000\2968491740.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name="./models/all-MiniLM-L6-v2")


In [13]:
vectorindex = FAISS.from_documents(text, model)

NameError: name 'text' is not defined

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex.as_retriever())

In [16]:
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B7FEEF1760>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B7FEEF2870>, model_name='llama-3.3-70b-versatile', temperature=0.9, model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['que

In [22]:
query = "High score scored by Virat kohli in any format(can give guess also)?"

langchain.debug=True

output = chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "High score scored by Virat kohli in any format(can give guess also)?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "possessed a broader array of shots,[47] Kohli's skills with the bat have earned him accolades from former West Indies great Brian Lara, who has ranked him among the best batsmen in the world alongside Joe Root, the England captain at the time.[48]",
      "question": "High score scored by Virat kohli in any format(can give guess also)?"
    },
    {
      "context": "References\n\n^ Lokapally 2016, p. 29.\n\n^ One Cricket (21 February 2024). \"Why Is Virat Kohli Called King Kohli?\". Retrieved 21 February 2024.\n\n^ \"See Who Is

In [27]:
print(output['answer'])

I don't know the high score scored by Virat Kohli in any format, as the provided text does not mention this information. However, I can suggest that Virat Kohli has scored at least 100 runs in multiple matches, including 50 ODI hundreds.




Multisource Rag pipeline

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [25]:
tool.run("machine Learning")

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
model = HuggingFaceEmbeddings(model_name="./models/all-MiniLM-L6-v2")

C:\Users\Apurav\AppData\Local\Temp\ipykernel_30556\2968491740.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name="./models/all-MiniLM-L6-v2")


In [5]:
loader = WebBaseLoader("https://langchain-ai.github.io/langgraph/tutorials/introduction/")

doc = loader.load()

In [22]:
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(doc)

chunks

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/introduction/', 'title': 'Learn the basics', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content='Learn the basics\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Learn the basics\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  Home\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n\n\n\n            \n  \n    Home\n  \n\n          \n\n\n\n\n\n   

In [47]:
vectordb = FAISS.from_documents(chunks, model)
vectordb

retriever = vectordb.as_retriever()

In [48]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000207F4517410>, search_kwargs={})

In [68]:
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool

def retriever_function(query):
    """Fetch relevant documents and format them correctly."""
    docs = retriever.get_relevant_documents(query)
    if not docs:
        return "No relevant documents found."
    
    return "\n\n".join([doc.page_content for doc in docs])  # Ensure clean text format


retriever_tool = Tool(
    name="FAISS retriever",
    func=retriever_function,
    description="use this to Retrieves documents from the FAISS datastore.",
)

In [62]:
retriever_tool.name

'FAISS retriever'

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [15]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [81]:
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate.from_template(
    """You are an AI assistant that follows a structured reasoning process.
    
    When responding, always format your answer as follows:
    
    Thought: Explain your reasoning
    Action: Choose a tool to use
    Action Input: Provide input for the tool
    
    If no tool is needed, respond with:
    Thought: Explanation
    Final Answer: Your response

    Question: {input}
    """
)


llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    temperature = 0.1,
    groq_api_key = "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",
)

tools = load_tools(["wikipedia"], llm=llm) + [arxiv, retriever_tool]

# tools_loaded = load_tools(["wikipedia"], llm = llm)

# all_tools = tools_loaded + tools

# all_tools


In [ ]:
from langchain.agents import initialize_agent, AgentType

# print(agent.agent.llm_chain.prompt.template)

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

agent.run("difference between machine learning and AI")




> Entering new AgentExecutor chain...
Thought: To answer this question, I need to understand the definitions and differences between machine learning and AI. I can start by looking up the general concepts of machine learning and AI on Wikipedia.

Action: wikipedia
Action Input: machine learning vs AI
Observation: Page: Explainable artificial intelligence
Summary: Explainable AI (XAI), often overlapping with interpretable AI, or explainable machine learning (XML), is a field of research within artificial intelligence (AI) that explores methods that provide humans with the ability of intellectual oversight over AI algorithms. The main focus is on the reasoning behind the decisions or predictions made by the AI algorithms, to make them more understandable and transparent. This addresses users' requirement to assess safety and scrutinize the automated decision making in applications. XAI counters the "black box" tendency of machine learning, where even the AI's designers cannot explain w

'The main difference between machine learning and AI is that machine learning is a subfield of AI that focuses on developing algorithms that can learn from data, while AI is a broader field that encompasses machine learning and other techniques to enable machines to perform tasks that typically require human intelligence.'